In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
motion = np.genfromtxt('../../../dataset4_wbc/live_extraction/step_0.06/gt/fwd_only/accelerations.csv', delimiter=',', dtype=np.float64)
contacts = np.genfromtxt('../../../dataset4_wbc/live_extraction/step_0.06/gt/fwd_only/accelerations.csv', delimiter=',', usecols=(57, 58, 59, 60), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  4281


In [6]:
for x in np.arange(0.1, 0.7, 0.1):
    print("Samples of ", x, ", ", len(np.where(np.isclose(motion[:, 1], x))[0]))

Samples of  0.1 ,  522
Samples of  0.2 ,  697
Samples of  0.30000000000000004 ,  893
Samples of  0.4 ,  880
Samples of  0.5 ,  705
Samples of  0.6 ,  513


In [7]:
for x in np.arange(0.1, 0.7, 0.1):
    print("Samples of ", x, ", ", len(np.where(np.isclose(X_relative_footsteps_accelerations_absolute[:, 0], x))[0]))

NameError: name 'X_relative_footsteps_accelerations_absolute' is not defined

In [8]:
len(motion)

4281

In [9]:
i = 0
accelerations_idx = []
while i < (len(motion) - 1):
    if motion[i, 1:4].tolist() != motion[i+1, 1:4].tolist():
        accelerations_idx.append(i)
        accelerations_idx.append(i+1)
    
    i += 1

accelerations_idx = np.array(accelerations_idx)

# Dataset Preparation

In [10]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [13]:
def absolute_footstep_accelerations_dataset(dataset, footsteps, debug=False):
    # Footsteps sizes
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 0:
            # Compute time difference between
            # curhent and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])

            if time_difference < 0.5:
                fl_moving = dataset[footsteps[idx], 8] > dataset[footsteps[idx-1], 8]
                fr_moving = dataset[footsteps[idx], 11] > dataset[footsteps[idx-1], 11]
                rl_moving = dataset[footsteps[idx], 14] > dataset[footsteps[idx-1], 14]
                rr_moving = dataset[footsteps[idx], 17] > dataset[footsteps[idx-1], 17]

                if debug:
                    print("Time difference: ", time_difference)
                    print("Footstep timestamp: ", dataset[footstep, 0])
                    print(f'lf moving: {fl_moving}. rf moving: {fr_moving}')
                    print(f'lh moving: {rl_moving}. rh moving: {rr_moving}')

                if (not fr_moving != fl_moving or not fr_moving == rl_moving or not fl_moving == rr_moving):
                    #print("Exception")
                    """
                    print("Time difference: ", time_difference)
                    #print("Footstep timestamp: ", dataset[footstep, 0])
                    print(f'fl moving: {fl_moving}. fr moving: {fr_moving}')
                    print(f'rl moving: {rl_moving}. rr moving: {rr_moving}')
                    """
                    continue
                
                assert fr_moving == rl_moving
                assert rr_moving == fl_moving
                assert fr_moving != fl_moving

                if (dataset[footsteps[idx], 44] - dataset[footsteps[idx-1], 44]) > 0:
                    inputs.append(dataset[footsteps[idx-1], 1:4].tolist() + 
                                  dataset[footsteps[idx-1], 51:57].tolist() + 
                                  dataset[footsteps[idx-1], 8:20].tolist() + 
                                  [fl_moving, fr_moving, rl_moving, rr_moving] +
                                  dataset[footsteps[idx], 1:4].tolist())
                    
                    if fr_moving and rl_moving:
                        labels.append([(dataset[footsteps[idx], 44] + dataset[footsteps[idx], 11]) - (dataset[footsteps[idx-1], 44] + dataset[footsteps[idx-1], 11]),
                                      (dataset[footsteps[idx], 45] + dataset[footsteps[idx], 12]) - (dataset[footsteps[idx-1], 45] + dataset[footsteps[idx-1], 12]),
                                      (dataset[footsteps[idx], 44] + dataset[footsteps[idx], 14]) - (dataset[footsteps[idx-1], 44] + dataset[footsteps[idx-1], 14]),
                                      (dataset[footsteps[idx], 45] + dataset[footsteps[idx], 15]) - (dataset[footsteps[idx-1], 45] + dataset[footsteps[idx-1], 15])])
                    
                    if fl_moving and rr_moving:
                        labels.append([(dataset[footsteps[idx], 44] + dataset[footsteps[idx], 8]) - (dataset[footsteps[idx-1], 44] + dataset[footsteps[idx-1], 8]),
                                      (dataset[footsteps[idx], 45] + dataset[footsteps[idx], 9]) - (dataset[footsteps[idx-1], 45] + dataset[footsteps[idx-1], 9]),
                                      (dataset[footsteps[idx], 44] + dataset[footsteps[idx], 17]) - (dataset[footsteps[idx-1], 44] + dataset[footsteps[idx-1], 17]),
                                      (dataset[footsteps[idx], 45] + dataset[footsteps[idx], 18]) - (dataset[footsteps[idx-1], 45] + dataset[footsteps[idx-1], 18])])
                
    # Convert to numpy array
    inputs_numpy = np.array(inputs)
    labels_numpy = np.array(labels)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_footsteps_accelerations_absolute, Y_footsteps_accelerations_absolute = absolute_footstep_accelerations_dataset(motion, np.array([x for x in range(len(motion))]))
#X_relative_footsteps_accelerations_absolute, Y_relative_footsteps_accelerations_absolute = absolute_footstep_accelerations_dataset(motion, accelerations_idx)

# Log
print(X_footsteps_accelerations_absolute.shape)
print(Y_footsteps_accelerations_absolute.shape)

(4084, 28)
(4084, 4)


In [14]:
%store X_footsteps_accelerations_absolute
%store Y_footsteps_accelerations_absolute

Stored 'X_footsteps_accelerations_absolute' (ndarray)
Stored 'Y_footsteps_accelerations_absolute' (ndarray)
